# Using Python Get Data From Baidu API

In [2]:
#coding=utf-8
import json
import geopandas
import matplotlib.pyplot as plt
import requests
from shapely.geometry import LineString
import time
import os

In [3]:
BASE_URL = 'https://restapi.amap.com/v3/traffic/status/rectangle?rectangle={loc}&key={k}&extensions=all'
keys = [
    '0f77fda9a61b3fa8ae32f99b2eabe534', 'ac644665b36b3ca24b4a7abe54661efb',
    'd718d4a3addce3439b3df02c0e8b599c',"2b239cb466670c8379747913be5ba79e"
]
Loc_of_Shanghai = '121.473658,31.230378'
key_flag = 3
Lines = []
Names = []
Speeds = []
Status = []
Lenghts = []
Lcodes = []
Timestamps = []

In [4]:
class Grid(object):
    def __init__(self, loc_all):
        self.loc_all = loc_all
        self.lng_sw = float(self.loc_all.split(',')[0])
        self.lat_sw = float(self.loc_all.split(',')[1])
        self.max = 0.06

    def get_max(self):
        for i in range(1, 10):
            data = str(self.lng_sw) + "," + str(self.lat_sw) + ';' + str(
                self.lng_sw + i / 100) + ',' + str(self.lat_sw + i / 100)
            print(i)
            url = BASE_URL.format(loc=data, k=keys[key_flag])
            print(url)
            res = requests.get(url)
            datas = res.json()
            if datas['status'] != '1':
                print(
                    "the maxium of distance is",
                    i,
                )
                break

    def get_grid(self):
        grid = []
        lng = []
        lat = []
        for i in range(-5,3):
            lng.append(self.lng_sw + self.max * i)
            lat.append(self.lat_sw + self.max * i)
        for i in range(8):
            for j in range(8):
                data = str(lng[i])+','+str(lat[j])+';'+str(lng[i]+self.max)+','+str(lat[j]+self.max)
                grid.append(data)
        return grid

In [5]:
loc = Grid(Loc_of_Shanghai)
locs= loc.get_grid()

In [6]:
#coding=utf-8

def request_map_one_minute():
    Times = time.strftime('%Y_%m_%d %H:%M:%S',time.localtime())
    global Lines, Names, Speeds, Status, Lenghts, Lcodes, Timestamps, locs, key
#     all_datas = []
    for i in locs:
        url = BASE_URL.format(loc=i, k=keys[3])
#         print(url)
        res = requests.get(url)
        datas = res.json()
#         all_datas.append(datas)
        if datas['status'] == '1':
            for road in datas['trafficinfo']['roads']:
                if 'speed' in road: 
                    s = int(road['speed']))
                else:
                    s = -1
                for code in road['lcodes']:
                    lcode.append[code]
                    Timestamps.append(Times)
                    Status.append(road['status'])
                    Speeds.append(s)
#                 polyline = road['polyline']
#                 polylines = [
#                     (float(y[0]), float(y[1])) for y in
#                     [x.split(',') for x in [i for i in polyline.split(';')]]
#                 ]
#                 line = LineString(polylines)
#                 Lines.append(line)
#                 Names.append(road['name'])
#                 Lcodes.append(road['lcodes'])
#                 Timestamps.append(Times)
#                 if 'speed' in road: Speeds.append(int(road['speed']))
#                 else: Speeds.append(100)
#                 Status.append(road['status'])
#                 Lenghts.append(line.length)

        else:
            if datas['infocode'] == '10003':
                key_flag = (key_flag + 1)%len(keys)
            if datas['infocode'] == '20003':
                continue
            else:
                print("Error of status",datas['infocode'])
#     with open("record.json","w") as f:
#         json.dump(all_datas,f)
#         print("加载入文件完成...")


def plot_show(pds_dict):
    ShangHai = geopandas.GeoDataFrame(pds_dict)
    #     ShangHai.to_file("shanghai.shp")
    #     ShangHai = geopandas.read_file("shanghai.shp")
    ShangHai.plot(column='speed', cmap='RdYlGn', legend=True)
    plt.figure(dpi=400)
    plt.show()


def save_to_file(data):
    datas = geopandas.GeoDataFrame(data)
    ti = "Shanghai_" + str(
        str(time.localtime()[1]) + '_' + str(time.localtime()[2]) + '_' +
        str(time.localtime()[3]))
    datas.to_file(ti,encoding='utf-8')

In [7]:
def print_ts(message):
    print ("[%s] %s"%(time.strftime("%Y-%m-%d %H:%M:%S", time.localtime()), message))

def run(interval, command):
    print_ts("Starting every %s seconds during 5:00 -- 22:00."%interval)
    global Lines,Names,Speeds,Status,Lenghts,Lcodes,Timestamps,locs,key
    while True:
        T_hour = time.localtime()[3]
        if T_hour >= 5 and T_hour <= 21:
            if time.localtime()[4] == 58 :
                datas = {'name':Names,'Timestamp':Timestamps,'Lcode':Lcodes,"speed":Speeds,'CongestionRate':Status,'lenght':Lenghts,"geometry":Lines}
                save_to_file(datas)
                print_ts("Finish Save to file")
                Start_hour = T_hour
                Lines.clear()
                Names.clear()
                Speeds.clear()
                Status.clear()
                Lenghts.clear()
                Lcodes.clear()
                Timestamps.clear()
            # sleep for the remaining seconds of interval
            # execute the command
            time_remaining = interval-time.time()%interval
            print_ts("Sleeping until %s (%s seconds)..."%((time.ctime(time.time()+time_remaining)), time_remaining))
            time.sleep(time_remaining)
            try :
                request_map_one_minute()
            except:
                print("Error")
            print_ts("Starting command.")
            print_ts("-"*100)
        else:
            time.sleep(time_remaining*command)
            
if __name__=="__main__":
    interval = 120
    command  = 100
    run(interval, command)

[2019-11-21 16:22:35] Starting every 120 seconds during 6:00 -- 21:00.
[2019-11-21 16:22:35] Sleeping until Thu Nov 21 16:24:00 2019 (84.24067211151123 seconds)...
[2019-11-21 16:24:10] Starting command.
[2019-11-21 16:24:10] ----------------------------------------------------------------------------------------------------
[2019-11-21 16:24:10] Sleeping until Thu Nov 21 16:26:00 2019 (109.6154088973999 seconds)...
[2019-11-21 16:26:08] Starting command.
[2019-11-21 16:26:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 16:26:08] Sleeping until Thu Nov 21 16:28:00 2019 (111.08794403076172 seconds)...
[2019-11-21 16:28:08] Starting command.
[2019-11-21 16:28:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 16:28:08] Sleeping until Thu Nov 21 16:30:00 2019 (111.25456523895264 seconds)...
[2019-11-21 16:30:09] Starting command.
[2019-11-21 16:30:09] ----

[2019-11-21 17:28:35] Starting command.
[2019-11-21 17:28:35] ----------------------------------------------------------------------------------------------------
[2019-11-21 17:28:35] Sleeping until Thu Nov 21 17:30:00 2019 (84.57986783981323 seconds)...
[2019-11-21 17:30:16] Starting command.
[2019-11-21 17:30:16] ----------------------------------------------------------------------------------------------------
[2019-11-21 17:30:16] Sleeping until Thu Nov 21 17:32:00 2019 (103.67317819595337 seconds)...
[2019-11-21 17:32:08] Starting command.
[2019-11-21 17:32:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 17:32:08] Sleeping until Thu Nov 21 17:34:00 2019 (111.22843790054321 seconds)...
[2019-11-21 17:34:16] Starting command.
[2019-11-21 17:34:16] ----------------------------------------------------------------------------------------------------
[2019-11-21 17:34:16] Sleeping until Thu Nov 21 17:36:00 2019 (103.

[2019-11-21 18:32:09] Starting command.
[2019-11-21 18:32:09] ----------------------------------------------------------------------------------------------------
[2019-11-21 18:32:09] Sleeping until Thu Nov 21 18:34:00 2019 (110.55492877960205 seconds)...
[2019-11-21 18:34:09] Starting command.
[2019-11-21 18:34:09] ----------------------------------------------------------------------------------------------------
[2019-11-21 18:34:09] Sleeping until Thu Nov 21 18:36:00 2019 (110.53804898262024 seconds)...
[2019-11-21 18:36:09] Starting command.
[2019-11-21 18:36:09] ----------------------------------------------------------------------------------------------------
[2019-11-21 18:36:09] Sleeping until Thu Nov 21 18:38:00 2019 (110.65592408180237 seconds)...
[2019-11-21 18:38:08] Starting command.
[2019-11-21 18:38:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 18:38:08] Sleeping until Thu Nov 21 18:40:00 2019 (111

[2019-11-21 19:36:08] Starting command.
[2019-11-21 19:36:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 19:36:08] Sleeping until Thu Nov 21 19:38:00 2019 (111.3742299079895 seconds)...
[2019-11-21 19:38:08] Starting command.
[2019-11-21 19:38:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 19:38:08] Sleeping until Thu Nov 21 19:40:00 2019 (111.37462997436523 seconds)...
[2019-11-21 19:40:08] Starting command.
[2019-11-21 19:40:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 19:40:08] Sleeping until Thu Nov 21 19:42:00 2019 (111.23578906059265 seconds)...
[2019-11-21 19:42:08] Starting command.
[2019-11-21 19:42:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 19:42:08] Sleeping until Thu Nov 21 19:44:00 2019 (111.

[2019-11-21 20:40:09] Starting command.
[2019-11-21 20:40:09] ----------------------------------------------------------------------------------------------------
[2019-11-21 20:40:09] Sleeping until Thu Nov 21 20:42:00 2019 (110.9675600528717 seconds)...
[2019-11-21 20:42:08] Starting command.
[2019-11-21 20:42:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 20:42:08] Sleeping until Thu Nov 21 20:44:00 2019 (111.95976400375366 seconds)...
[2019-11-21 20:44:08] Starting command.
[2019-11-21 20:44:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 20:44:08] Sleeping until Thu Nov 21 20:46:00 2019 (111.21057295799255 seconds)...
[2019-11-21 20:46:08] Starting command.
[2019-11-21 20:46:08] ----------------------------------------------------------------------------------------------------
[2019-11-21 20:46:08] Sleeping until Thu Nov 21 20:48:00 2019 (111.

...

[2019-11-22 11:58:08] Starting command.
[2019-11-22 11:58:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 11:58:08] Sleeping until Fri Nov 22 12:00:00 2019 (111.27800703048706 seconds)...
[2019-11-22 12:00:08] Starting command.
[2019-11-22 12:00:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 12:00:25] Finish Save to file
[2019-11-22 12:00:25] Sleeping until Fri Nov 22 12:02:00 2019 (94.32705688476562 seconds)...
[2019-11-22 12:02:08] Starting command.
[2019-11-22 12:02:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 12:02:08] Sleeping until Fri Nov 22 12:04:00 2019 (111.29472398757935 seconds)...
[2019-11-22 12:04:08] Starting command.
[2019-11-22 12:04:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 12:04:08] Sle

[2019-11-22 13:02:09] Starting command.
[2019-11-22 13:02:09] ----------------------------------------------------------------------------------------------------
[2019-11-22 13:02:09] Sleeping until Fri Nov 22 13:04:00 2019 (110.94041013717651 seconds)...
[2019-11-22 13:04:09] Starting command.
[2019-11-22 13:04:09] ----------------------------------------------------------------------------------------------------
[2019-11-22 13:04:09] Sleeping until Fri Nov 22 13:06:00 2019 (110.09886598587036 seconds)...
[2019-11-22 13:06:08] Starting command.
[2019-11-22 13:06:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 13:06:08] Sleeping until Fri Nov 22 13:08:00 2019 (111.647047996521 seconds)...
[2019-11-22 13:08:16] Starting command.
[2019-11-22 13:08:16] ----------------------------------------------------------------------------------------------------
[2019-11-22 13:08:16] Sleeping until Fri Nov 22 13:10:00 2019 (103.3

[2019-11-22 14:06:09] Starting command.
[2019-11-22 14:06:09] ----------------------------------------------------------------------------------------------------
[2019-11-22 14:06:09] Sleeping until Fri Nov 22 14:08:00 2019 (110.84958481788635 seconds)...
[2019-11-22 14:08:08] Starting command.
[2019-11-22 14:08:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 14:08:08] Sleeping until Fri Nov 22 14:10:00 2019 (111.97893190383911 seconds)...
[2019-11-22 14:10:08] Starting command.
[2019-11-22 14:10:08] ----------------------------------------------------------------------------------------------------
[2019-11-22 14:10:08] Sleeping until Fri Nov 22 14:12:00 2019 (111.37531900405884 seconds)...
[2019-11-22 14:12:09] Starting command.
[2019-11-22 14:12:09] ----------------------------------------------------------------------------------------------------
[2019-11-22 14:12:09] Sleeping until Fri Nov 22 14:14:00 2019 (110

Error
[2019-11-22 15:10:00] Starting command.
[2019-11-22 15:10:00] ----------------------------------------------------------------------------------------------------
[2019-11-22 15:10:00] Sleeping until Fri Nov 22 15:12:00 2019 (119.8250150680542 seconds)...
Error
[2019-11-22 15:12:00] Starting command.
[2019-11-22 15:12:00] ----------------------------------------------------------------------------------------------------
[2019-11-22 15:12:00] Sleeping until Fri Nov 22 15:14:00 2019 (119.88291883468628 seconds)...
Error
[2019-11-22 15:14:00] Starting command.
[2019-11-22 15:14:00] ----------------------------------------------------------------------------------------------------
[2019-11-22 15:14:00] Sleeping until Fri Nov 22 15:16:00 2019 (119.91817498207092 seconds)...
Error
[2019-11-22 15:16:00] Starting command.
[2019-11-22 15:16:00] ----------------------------------------------------------------------------------------------------
[2019-11-22 15:16:00] Sleeping until Fri No

ValueError: Cannot write empty DataFrame to file.

In [ ]:
#coding=utf-8
